In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from models.venezia import Venezia
from utils.model_controler import freeze_model, run_train_test_2_input
from utils.custom_logger import get_logger


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
touch = 'softness'
train = torch.load(f'./dataset/{touch}_train.pth')
test = torch.load(f'./dataset/{touch}_test.pth')

In [ ]:
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

In [ ]:
#model = load_venezia_pretrain()
model = Venezia()
model.load_state_dict(torch.load(f'.checkpoints/best/best_venezia.pth'))

In [ ]:
freeze_model(model, exception=
             ['classifier', 'polling', 'input_preprocessor']) # modality_head

model.to(device)

In [ ]:
# 손실 함수 및 최적화 함수 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5
test_step = 1
logger = get_logger('venezia_stable')

In [ ]:
logger.info('start_new_run--------------------------------')

result = run_train_test_2_input(
    model=model, 
    train_loader=train_loader, 
    test_loader=test_loader,
    best_test_loss = float('inf'),
    criterion=criterion, 
    optimizer=optimizer, 
    num_epochs=num_epochs, 
    test_step=test_step, 
    logger = logger, 
    device = device,
    path = f'.checkpoints/best/best_venezia_{touch}.pth')